In [9]:
import os
import json
from dotenv import load_dotenv
from typing import Dict, Any
from datetime import datetime

import requests
import pandas as pd
import yfinance as yf

load_dotenv()
API_KEY = os.getenv("VANTAGE_API_KEY")

## Data Fetching and Caching

In [10]:
class APIRateLimitError(Exception):
    """Custom exception raised when the API rate limit is exceeded."""
    pass

def fetch_income_statement(symbol: str, api_key: str, folder: str = 'data') -> dict:
    file_path = os.path.join(folder, f'{symbol}_income_statement.json')
    if os.path.exists(file_path):
        with open(file_path, 'r') as f:
            return json.load(f)
    url = f'https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={symbol}&apikey={api_key}'
    response = requests.get(url)
    data = response.json()
    if "Information" in data and "rate limit" in data["Information"].lower():
        raise APIRateLimitError(data["Information"])
    os.makedirs(folder, exist_ok=True)
    with open(file_path, 'w') as f:
        json.dump(data, f)
    return data


def fetch_balance_sheet(symbol: str, api_key: str, folder: str = 'data') -> dict:
    file_path = os.path.join(folder, f'{symbol}_balance_sheet.json')
    if os.path.exists(file_path):
        with open(file_path, 'r') as f:
            return json.load(f)
    url = f'https://www.alphavantage.co/query?function=BALANCE_SHEET&symbol={symbol}&apikey={api_key}'
    response = requests.get(url)
    data = response.json()
    if "Information" in data and "rate limit" in data["Information"].lower():
        raise APIRateLimitError(data["Information"])
    os.makedirs(folder, exist_ok=True)
    with open(file_path, 'w') as f:
        json.dump(data, f)
    return data


def fetch_cash_flow(symbol: str, api_key: str, folder: str = 'data') -> dict:
    file_path = os.path.join(folder, f'{symbol}_cash_flow.json')
    if os.path.exists(file_path):
        with open(file_path, 'r') as f:
            return json.load(f)
    url = f'https://www.alphavantage.co/query?function=CASH_FLOW&symbol={symbol}&apikey={api_key}'
    response = requests.get(url)
    data = response.json()
    if "Information" in data and "rate limit" in data["Information"].lower():
        raise APIRateLimitError(data["Information"])
    os.makedirs(folder, exist_ok=True)
    with open(file_path, 'w') as f:
        json.dump(data, f)
    return data

## Merge Reports

In [11]:
def convert_columns_to_numeric(df):
    """
    Convert necessary columns to numeric within a DataFrame.
    """
    numeric_cols = [
        'netIncome', 'totalRevenue', 'operatingCashflow', 'capitalExpenditures',
        'cashAndCashEquivalentsAtCarryingValue', 'totalAssets',
        'totalCurrentLiabilities', 'currentDebt', 'cashAndShortTermInvestments',
        'longTermDebt', 'commonStockSharesOutstanding'
    ]
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')


def merge_reports_df(income_statement: dict,
                     balance_sheet: dict,
                     cash_flow: dict) -> Dict[str, Any]:
    """
    Merge annual and quarterly reports from three API responses into two DataFrames.
    Return a dict with 'symbol', 'annual' DataFrame, and 'quarterly' DataFrame.
    """
    # --- Annual Reports ---
    annual_income = pd.DataFrame(income_statement.get('annualReports', []))
    annual_balance = pd.DataFrame(balance_sheet.get('annualReports', []))
    annual_cash = pd.DataFrame(cash_flow.get('annualReports', []))

    # Drop unwanted columns to avoid collisions in merges
    annual_balance.drop(columns=['reportedCurrency'], errors='ignore', inplace=True)
    annual_cash.drop(columns=['reportedCurrency', 'netIncome'], errors='ignore', inplace=True)

    # Convert 'fiscalDateEnding' to datetime and sort
    for df in (annual_income, annual_balance, annual_cash):
        if 'fiscalDateEnding' in df.columns:
            df['fiscalDateEnding'] = pd.to_datetime(df['fiscalDateEnding'])
            df.sort_values('fiscalDateEnding', inplace=True)
            df.reset_index(drop=True, inplace=True)

    # Merge annual data
    annual_df = pd.merge(annual_income, annual_balance, on='fiscalDateEnding', how='outer')
    annual_df = pd.merge(annual_df, annual_cash, on='fiscalDateEnding', how='outer')
    convert_columns_to_numeric(annual_df)

    # --- Quarterly Reports ---
    quarterly_income = pd.DataFrame(income_statement.get('quarterlyReports', []))
    quarterly_balance = pd.DataFrame(balance_sheet.get('quarterlyReports', []))
    quarterly_cash = pd.DataFrame(cash_flow.get('quarterlyReports', []))

    # Drop unwanted columns
    quarterly_balance.drop(columns=['reportedCurrency'], errors='ignore', inplace=True)
    quarterly_cash.drop(columns=['reportedCurrency', 'netIncome'], errors='ignore', inplace=True)

    for df in (quarterly_income, quarterly_balance, quarterly_cash):
        if 'fiscalDateEnding' in df.columns:
            df['fiscalDateEnding'] = pd.to_datetime(df['fiscalDateEnding'])
            df.sort_values('fiscalDateEnding', inplace=True)
            df.reset_index(drop=True, inplace=True)

    # Merge quarterly data
    quarterly_df = pd.merge(quarterly_income, quarterly_balance, on='fiscalDateEnding', how='outer')
    quarterly_df = pd.merge(quarterly_df, quarterly_cash, on='fiscalDateEnding', how='outer')
    convert_columns_to_numeric(quarterly_df)

    return {
        'symbol': income_statement.get('symbol', 'N/A'),
        'annual': annual_df,
        'quarterly': quarterly_df
    }

## Price Retrieval

In [12]:
def get_yearly_close_prices(ticker_symbol: str,
                            start_year: int,
                            end_year: int) -> pd.DataFrame:
    """
    Retrieve and return a DataFrame with columns:
        'year' and 'close' (the last close price of that year).
    """
    start_date = f"{start_year}-01-01"
    end_date = f"{end_year}-12-31"

    ticker = yf.Ticker(ticker_symbol)
    hist = ticker.history(start=start_date, end=end_date)

    if hist.empty:
        return pd.DataFrame(columns=['year', 'close'])

    if not isinstance(hist.index, pd.DatetimeIndex):
        hist.index = pd.to_datetime(hist.index)

    # Resample to get the last closing price of each year
    yearly = hist['Close'].resample('YE').last().dropna()

    # Convert to a DataFrame with columns: year, close
    df_price = yearly.reset_index()
    df_price['year'] = df_price['Date'].dt.year
    df_price.rename(columns={'Close': 'close'}, inplace=True)
    return df_price[['year', 'close']]


def get_latest_price(ticker_symbol: str) -> float:
    """
    Get the latest market price (attempt from 'regularMarketPrice'
    or fallback to 'previousClose').
    """
    ticker = yf.Ticker(ticker_symbol)
    latest_price = ticker.info.get('previousClose')
    return float(latest_price) if latest_price else None

## Analysis

In [13]:
def analyze_dataframes(symbol: str,
                       annual_df: pd.DataFrame,
                       quarterly_df: pd.DataFrame,
                       num_periods: int = 5) -> Dict[str, Any]:
    """
    Perform all the computations on annual_df and quarterly_df in a vectorized manner,
    then return a dictionary with the final metrics.
    """

    # ----------------------------------------
    # 1. Prepare the DataFrames
    # ----------------------------------------
    # For annual: create columns for FCF, yoy growth
    annual_df['fcf'] = annual_df['operatingCashflow'] - annual_df['capitalExpenditures']
    annual_df['year'] = annual_df['fiscalDateEnding'].dt.year

    # For TTM calculations in quarterly_df, use rolling(4) sums
    quarterly_df['fcf'] = quarterly_df['operatingCashflow'] - quarterly_df['capitalExpenditures']
    quarterly_df['ttm_fcf'] = quarterly_df['fcf'].rolling(4).sum()
    quarterly_df['ttm_net_income'] = quarterly_df['netIncome'].rolling(4).sum()

    # We also want the shares from the latest quarter (assuming it doesn't drastically change).
    # We'll just take the 'commonStockSharesOutstanding' from the latest quarter for TTM calcs.
    # For TTM EPS or TTM FCF/share, we assume the most recent share count is a decent approximation.
    quarterly_df['shares'] = quarterly_df['commonStockSharesOutstanding'].ffill()

    # TTM EPS = ttm_net_income / shares
    quarterly_df['ttm_eps'] = quarterly_df.apply(
        lambda row: row['ttm_net_income'] / row['shares']
        if pd.notnull(row['ttm_net_income']) and pd.notnull(row['shares']) and row['shares'] != 0
        else None,
        axis=1
    )

    # TTM FCF per share = ttm_fcf / shares
    quarterly_df['ttm_fcf_ps'] = quarterly_df.apply(
        lambda row: row['ttm_fcf'] / row['shares']
        if pd.notnull(row['ttm_fcf']) and pd.notnull(row['shares']) and row['shares'] != 0
        else None,
        axis=1
    )

    # Latest row in quarterly_df (most recent quarter)
    latest_q = quarterly_df.iloc[-1] if not quarterly_df.empty else None

    # ----------------------------------------
    # 2. Basic Valuation Metrics (TTM, Latest EPS)
    # ----------------------------------------
    latest_quarter_eps = None
    ttm_eps = None
    ttm_fcf_ps = None
    fiscal_date_ending = None

    if latest_q is not None:
        net_income_latest = latest_q['netIncome']
        shares_latest = latest_q['commonStockSharesOutstanding']
        # EPS for that single quarter
        if pd.notnull(net_income_latest) and pd.notnull(shares_latest) and shares_latest != 0:
            latest_quarter_eps = net_income_latest / shares_latest
        # TTM
        ttm_eps = latest_q['ttm_eps']
        ttm_fcf_ps = latest_q['ttm_fcf_ps']
        if pd.notnull(latest_q['fiscalDateEnding']):
            fiscal_date_ending = latest_q['fiscalDateEnding'].strftime('%Y-%m-%d')

    # ----------------------------------------
    # 3. Annual Growth Metrics
    # ----------------------------------------
    # We'll select the last (num_periods + 1) to compute average yoy growth
    # Because yoy growth uses shift(1).
    df_for_growth = annual_df.copy()
    df_for_growth['revenue_growth'] = df_for_growth['totalRevenue'].pct_change()
    df_for_growth['fcf_growth'] = df_for_growth['fcf'].pct_change()
    df_for_growth['shares_growth'] = df_for_growth['commonStockSharesOutstanding'].pct_change()

    # For ROCE using FCF, we define capital employed = totalAssets - totalCurrentLiabilities
    df_for_growth['capital_employed'] = df_for_growth['totalAssets'] - df_for_growth['totalCurrentLiabilities']
    df_for_growth['roce_fcf'] = df_for_growth.apply(
        lambda row: row['fcf'] / row['capital_employed']
        if pd.notnull(row['fcf']) and pd.notnull(row['capital_employed']) and row['capital_employed'] != 0
        else None,
        axis=1
    )

    # net debt = (currentDebt + longTermDebt) - cash
    # We unify 'cash' column first (prefer `cashAndShortTermInvestments`).
    df_for_growth['cash'] = df_for_growth.get('cashAndShortTermInvestments', 0).fillna(0)
    # If there's also `cashAndCashEquivalentsAtCarryingValue` we can combine or fill
    if 'cashAndCashEquivalentsAtCarryingValue' in df_for_growth.columns:
        df_for_growth['cash'] = df_for_growth['cash'].where(
            df_for_growth['cash'] != 0,
            df_for_growth['cashAndCashEquivalentsAtCarryingValue']
        )

    df_for_growth['debt'] = df_for_growth['currentDebt'] + df_for_growth['longTermDebt']
    df_for_growth['net_debt'] = df_for_growth['debt'] - df_for_growth['cash']
    df_for_growth['net_debt_to_fcf'] = df_for_growth.apply(
        lambda row: row['net_debt'] / row['fcf']
        if pd.notnull(row['net_debt']) and pd.notnull(row['fcf']) and row['fcf'] != 0
        else None,
        axis=1
    )

    # FCF margin
    df_for_growth['fcf_margin'] = df_for_growth.apply(
        lambda row: row['fcf'] / row['totalRevenue']
        if pd.notnull(row['fcf']) and pd.notnull(row['totalRevenue']) and row['totalRevenue'] != 0
        else None,
        axis=1
    )

    # Select the last num_periods for averaging
    recent_subset = df_for_growth.tail(num_periods)

    avg_turnover_growth = recent_subset['revenue_growth'].mean() if not recent_subset.empty else None
    avg_fcf_growth = recent_subset['fcf_growth'].mean() if not recent_subset.empty else None
    avg_stock_increase = recent_subset['shares_growth'].mean() if not recent_subset.empty else None
    avg_roce_fcf = recent_subset['roce_fcf'].mean() if not recent_subset.empty else None
    latest_net_debt_to_fcf = df_for_growth['net_debt_to_fcf'].iloc[-1] if not df_for_growth.empty else None
    avg_fcf_margin = recent_subset['fcf_margin'].mean() if not recent_subset.empty else None

    # ----------------------------------------
    # 4. Price Data & Ratios (Annual)
    # ----------------------------------------
    # We get up to 10 years of data to cover the user's range
    end_year = datetime.today().year - 1  # last complete year
    start_year = end_year - 10
    df_price = get_yearly_close_prices(symbol, start_year, end_year)

    # Merge annual data with price data on 'year'
    annual_price_df = pd.merge(annual_df, df_price, on='year', how='left')

    # Compute annual EPS
    annual_price_df['eps'] = annual_price_df.apply(
        lambda row: row['netIncome'] / row['commonStockSharesOutstanding']
        if pd.notnull(row['netIncome']) and pd.notnull(row['commonStockSharesOutstanding']) and row['commonStockSharesOutstanding'] != 0
        else None,
        axis=1
    )

    # Price / EPS
    annual_price_df['p_e'] = annual_price_df.apply(
        lambda row: row['close'] / row['eps']
        if pd.notnull(row['close']) and pd.notnull(row['eps']) and row['eps'] > 0
        else None,
        axis=1
    )

    # FCF per share
    annual_price_df['fcf_per_share'] = annual_price_df.apply(
        lambda row: row['fcf'] / row['commonStockSharesOutstanding']
        if pd.notnull(row['fcf']) and pd.notnull(row['commonStockSharesOutstanding']) and row['commonStockSharesOutstanding'] != 0
        else None,
        axis=1
    )

    # Price / FCF
    annual_price_df['p_fcf'] = annual_price_df.apply(
        lambda row: row['close'] / row['fcf_per_share']
        if pd.notnull(row['close']) and pd.notnull(row['fcf_per_share']) and row['fcf_per_share'] > 0
        else None,
        axis=1
    )

    # Filter to non-null p_e / p_fcf
    valid_pe = annual_price_df.dropna(subset=['p_e'])
    valid_pfcf = annual_price_df.dropna(subset=['p_fcf'])

    # Compute average P/E and P/FCF over last 5/10 years (or whatever is possible)
    pe_values = valid_pe['p_e'].values
    pfcf_values = valid_pfcf['p_fcf'].values

    average_pe_5y = None
    average_pe_10y = None
    if len(pe_values) >= 5:
        average_pe_5y = pe_values[-5:].mean()
    if len(pe_values) >= 10:
        average_pe_10y = pe_values[-10:].mean()

    average_pfcf_5y = None
    average_pfcf_10y = None
    if len(pfcf_values) >= 5:
        average_pfcf_5y = pfcf_values[-5:].mean()
    if len(pfcf_values) >= 10:
        average_pfcf_10y = pfcf_values[-10:].mean()

    # We'll also compute an average P/E for the last num_periods specifically
    # (this is the "average_per" in your old code).
    last_n_pe = valid_pe.tail(num_periods)['p_e']
    average_per = last_n_pe.mean() if not last_n_pe.empty else None

    # Now get latest price from Yahoo
    latest_price = get_latest_price(symbol)

    # TTM PER (latest price / TTM EPS)
    ttm_per = None
    if latest_price and ttm_eps and ttm_eps > 0:
        ttm_per = latest_price / ttm_eps

    # Latest PER (annualized from last quarter's EPS * 4)
    latest_per = None
    if latest_price and latest_quarter_eps and (latest_quarter_eps * 4) > 0:
        latest_per = latest_price / (latest_quarter_eps * 4)

    # ----------------------------------------
    # 5. DCF Inputs from the latest annual row
    # ----------------------------------------
    latest_annual = annual_df.iloc[-1] if not annual_df.empty else None

    dcf_inputs = {}
    if latest_annual is not None:
        # Combine possible cash columns
        cash_val = latest_annual.get('cashAndShortTermInvestments', 0)
        if not cash_val:
            cash_val = latest_annual.get('cashAndCashEquivalentsAtCarryingValue', 0)

        current_debt = latest_annual.get('currentDebt') or 0
        long_term_debt = latest_annual.get('longTermDebt') or 0
        debt_total = current_debt + long_term_debt

        shares = latest_annual.get('commonStockSharesOutstanding') or 0

        dcf_inputs = {
            'fcf_ps': float(ttm_fcf_ps),
            'cash': float(cash_val) if cash_val else None,
            'debt': float(debt_total),
            'shares': float(shares),
        }

    # ----------------------------------------
    # 6. Compile Results into Dictionary
    # ----------------------------------------
    results = {
        # Growth & Profitability
        'avg_turnover_growth': avg_turnover_growth,
        'avg_FCF_growth': avg_fcf_growth,
        'avg_ROCE_using_FCF': avg_roce_fcf,
        'latest_net_debt_to_FCF': latest_net_debt_to_fcf,
        'avg_stock_increase': avg_stock_increase,
        'avg_FCF_margin': avg_fcf_margin,

        # Valuation (P/E, P/FCF)
        'latest_price': latest_price,
        'ttm_per': ttm_per,
        'latest_per': latest_per,
        'average_per': average_per,

        'average_pe_5y': average_pe_5y,
        'average_pe_10y': average_pe_10y,
        'average_pfcf_5y': average_pfcf_5y,
        'average_pfcf_10y': average_pfcf_10y,

        # TTM / Quarter-based metrics
        'latest_quarterly_eps': latest_quarter_eps,
        'ttm_eps': ttm_eps,
        'ttm_fcf_ps': ttm_fcf_ps,
        'fiscal_date_ending': fiscal_date_ending,

        # DCF Inputs
        'dcf_inputs': dcf_inputs,

        # Additional metadata
        'analysis_date': datetime.now().strftime('%Y-%m-%d'),
        'symbol': symbol
    }

    return results

## High-Level Function

In [14]:
def analyze_stock(ticker: str, api_key: str, num_periods: int = 5) -> Dict[str, Any]:
    """
    Fetch raw data, merge, run the all-in-DataFrame analysis, and return
    a dictionary of results (no printing or saving).
    """
    # Fetch data
    income_statement = fetch_income_statement(ticker, api_key)
    balance_sheet_data = fetch_balance_sheet(ticker, api_key)
    cash_flow_data = fetch_cash_flow(ticker, api_key)

    # Merge into DataFrames
    merged_data = merge_reports_df(income_statement, balance_sheet_data, cash_flow_data)
    annual_df = merged_data['annual']
    quarterly_df = merged_data['quarterly']
    symbol = merged_data.get('symbol', ticker)

    # Perform the big analysis
    results = analyze_dataframes(symbol, annual_df, quarterly_df, num_periods=num_periods)
    return results

## Usage

In [15]:
ticker = 'AMZN'

In [16]:
metrics_dict = analyze_stock(ticker, API_KEY, num_periods=5)

# Ensure the analysis directory exists
os.makedirs('analysis', exist_ok=True)

# Save the metrics_dict to a JSON file
file_path = os.path.join('analysis', f'{ticker}.json')
with open(file_path, 'w') as f:
    json.dump(metrics_dict, f, indent=4)

print(json.dumps(metrics_dict, indent=4))

0              NaN
1     2.920000e+09
2     2.516000e+09
3     2.092000e+09
4     3.950000e+08
5     2.031000e+09
6     1.949000e+09
7     7.450000e+09
8     9.399000e+09
9     6.410000e+09
10    1.729600e+10
11    2.165300e+10
12    2.592400e+10
13   -1.472600e+10
14   -1.689300e+10
15    3.221700e+10
Name: fcf, dtype: float64
0    2008-12-31
1    2009-12-31
2    2010-12-31
3    2011-12-31
4    2012-12-31
5    2013-12-31
6    2014-12-31
7    2015-12-31
8    2016-12-31
9    2017-12-31
10   2018-12-31
11   2019-12-31
12   2020-12-31
13   2021-12-31
14   2022-12-31
15   2023-12-31
Name: fiscalDateEnding, dtype: datetime64[ns]
{
    "avg_turnover_growth": 0.2024668391876207,
    "avg_FCF_growth": -0.7757712410135389,
    "avg_ROCE_using_FCF": 0.05429950098660301,
    "latest_net_debt_to_FCF": -0.2815904646615141,
    "avg_stock_increase": 3.834359903448617,
    "avg_FCF_margin": 0.027382854427784643,
    "latest_price": 242.06,
    "ttm_per": 51.020547445255474,
    "latest_per": 41.49746